# Prerequistes

You need to have Dallinger and Griduniverse installed. In addition, you need a data file
to analyse and you need to know the date/times you want to view it at.

A simple example is `6195d89e-c2c0-c65c-8f09-ba0f27143b35` which has meaningful data from
`2018-04-20 16:32:04` until `2018-04-20 16:32:22` but any other will work.

You also need to ensure that the `dallinger-import` database exists and is writable by
the dallinger user.

# Use

Simply run all cells to get the notebook in a working state. Then click the advance button
below the grid to advance the time by 1 second.

# Caveats

This is a work in progress, some things do not work as intended. Specifically:

* The widget loses which tab is selected on refresh
* You can only progress forward in time, and only from a known-good point in the history
* You need to re-enter the experiment config as part of the `restore_state_from_replay` call
* Not all Griduniverse features are supported
* Older Griduniverse exports do not display chat history
* The teardown code is not working correctly. You will not be able to run cell 1 twice 
  within the same kernel. You must shut down your jupyter notebook server and restart
  in order to run it again.

In [ ]:
from dlgr.griduniverse.experiment import Griduniverse
from dallinger import db
from dallinger.config import get_config

experiment = Griduniverse()
participants = 3
import datetime


replay = experiment.restore_state_from_replay(
    '6195d89e-c2c0-c65c-8f09-ba0f27143b35',
    session=db.init_db(drop_all=True),
    max_participants=participants,
    num_dynos_worker=participants,
    walls_density = 0.50,
    walls_contiguity = 0.90,
    rows = 25,
    columns = 25,
)
# This time is taken from the logs of the experiment, it is the start of the experiment
target = datetime.datetime(2018, 4, 20, 16, 32, 4)
# This unusual syntax is to allow for multiple jupyter cells to interact with the experiment
# without it being torn down. You will need to restart your jupyter session to make the teardown
# happen
scrubber = replay.__enter__()
scrubber(target)


In [ ]:
experiment.widget.render()
display(experiment.widget)

In [ ]:
from ipywidgets import widgets

# Create a button that advances the time in the experiment by 1 second and re-renders the output widget
advance_button = widgets.Button(
    description='Advance',
    disabled=False,
    button_style='success',
    tooltip='Advance the replay by 1 second',
    icon='clock'
)
def advance(button):
    global target
    target = target + datetime.timedelta(seconds=1)
    scrubber(target)
    experiment.widget.render()
advance_button.on_click(advance)
display(advance_button)